In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns

plt.style.use('seaborn')
sns.set(font_scale=2.5)

import missingno as msno    # 누락값 표시
import warnings
warnings.filterwarnings('ignore')  # 워닝 무시

# 새로운 창 안 뜨고 노트북에 바로 뜸
%matplotlib inline 

In [ ]:
data_wanted = pd.read_csv('./data/data_wanted.csv', encoding='euc-kr')

In [ ]:
data_programmers

data_wanted

## 데이터 전처리

### Wanted

In [ ]:
# Location에서 '.한국' 제거

data_wanted1 = data_wanted
data_wanted1["Location"] = list(map(lambda i: i.replace(".한국", ""), data_wanted["Location"]))

In [ ]:
# Position keyword 확인
from collections import Counter

arr = [i.split() for i in data_wanted1.Position]

keywords = [j for i in arr for j in i]

# Counter(keywords)
list(data_wanted1.Position)

In [ ]:
# Position 키워드에 따라 데이터 drop

keywords = ['프론트', '프런트', '백', '앤드', '엔드', '풀', '스택', '웹', 'Front', 'Back', 'end',
            'S/W', 'Software', 'SW', '소프트웨어', 'React', 'Python', 'Java'] 
            
mask = data_wanted1['Position'].str.contains('|'.join(keywords), case = False)

filtered_data = data_wanted1[mask]
list(filtered_data.Position)

In [ ]:
# 'Skill' 채우기 : 'Position' 값에서 추출

# 'Position'에서 키워드 선정 
filtered_data[filtered_data["Skill"].isna()].Position
keywords = {'C++':"C", 'JS':"Java Script", 'Python':"Python", 'Node':"Node", 'React':"React"}

mask = filtered_data[filtered_data["Skill"].isna()]

for i in mask["Position"]:
    for j in keywords.keys():
        if j.casefold() in i.casefold():
            mask.loc[mask["Position"] == i, "Skill"] = keywords[j]

filtered_data.loc[filtered_data["Skill"].isna(), "Skill"] = mask["Skill"]

In [ ]:
# 'Skill' 채우기 후 NaN 값 확인
for col in filtered_data:
    msg = 'column: {:>10}\t Number of NaN value: {:.0f}'.format(col, filtered_data[col].isnull().sum())
    print(msg)

filtered_data[filtered_data["Skill"].isna()]

# Skill이 NaN인 row drop
filtered_data = filtered_data.drop(index=filtered_data[filtered_data['Skill'].isna()].index, axis=1)

In [ ]:
# 'Position' [웹 풀스택, 서버/백엔드, 프론트엔드]로 분류
list(filtered_data.Location)


### Programmers

In [ ]:
data_programmers = pd.read_csv('./data/data_programmers.csv', encoding='euc-kr')

In [ ]:
# 'Position' [웹 풀스택, 서버/백엔드, 프론트엔드]로 분류
arr = list(map(lambda j: j[0], map(lambda i: i.split(), data_programmers.Position)))
arr = list(map(lambda i: i.replace("웹", "풀스택"), arr))

data_programmers1 = data_programmers
data_programmers1['Position'] = arr

In [ ]:
# 'Skill'  NaN 값 확인
for col in data_programmers1:
    msg = 'column: {:>10}\t Number of NaN value: {:.0f}'.format(col, data_programmers1[col].isnull().sum())
    print(msg)

data_programmers1[data_programmers1["Skill"].isna()]

# Skill이 NaN인 row drop(15 rows)
data_programmers1 = data_programmers1.drop(index=data_programmers1[data_programmers1['Skill'].isna()].index, axis=1)

In [ ]:
# 'Location' 정리
list(data_programmers1.Location)
keywords = ['서울', '경기', '부산', '대전', '대구']
location = data_programmers1.Location
for i in keywords:
    for j in location:
        if i in j:
            j = i
            break

## 데이터 시각화

In [ ]:
# 기술 스택별 공고 갯수
group_skill = data_filled_sk["Company"].groupby(data_filled_sk["Skill"]).count()
print(group_skill)

# 지역별 공고 갯수
group_loc = data_filled_sk["Company"].groupby(data_filled_sk["Location"]).count()
print(group_loc)